### Imports & Setting

In [1]:
## setting
options(repr.plot.width = 15, repr.plot.height = 8)
setwd("~/TS2024/data/Time Series Data/")

### 1번 문제

In [ ]:
##----------(6) acf/pacf plot----------
data1 = list(acf = as.numeric(strsplit("-0.48 -0.04 -0.06 0.14 -0.22 0.19 -0.10 -0.02 0.09 -0.03 -0.12 0.09 0.03", " ")[[1]]),
             pacf = as.numeric(strsplit("-0.48 -0.24 -0.21 0.01 -0.20 -0.01 -0.05 -0.15 -0.04 -0.06 -0.04 0.02 0.06", " ")[[1]]))
data2 = list(acf = as.numeric(strsplit("0.59 0.44 0.33 0.23 0.24 0.16 0.05 0.01 -0.03 -0.11 -0.08 -0.07 0.01", " ")[[1]]),
             pacf = as.numeric(strsplit("0.59 0.13 0.04 -0.02 0.12 -0.05 -0.12 -0.03 -0.01 -0.13 0.06 0.04 0.13", " ")[[1]]))
data3 = list(acf = as.numeric(strsplit("-0.44 -0.05 -0.01 -0.03 0.12 -0.15 0.15 -0.04 -0.10 0.09 0.08 -0.07 0.06", " ")[[1]]),
             pacf = as.numeric(strsplit("-0.44 -0.31 -0.25 -0.25 -0.07 -0.21 -0.01 0.02 -0.09 -0.02 0.03 -0.02 0.01", " ")[[1]]))
data = list(data1, data2, data3)

## plotting 3 plots
for (i in 1:3) {
    par(mfrow = c(1,2))
    plot(x = 1:13, y = data[[i]]$acf, type = "n", ylim = c(-1, 1), main = paste("data", i,"ACF"), xlab = "lag", ylab = "ACF")
    abline(h = 0)
    segments(x0 = 1:13, y0 = 0, x1 = 1:13, y1 = data[[i]]$acf)
    abline(h = 0.2, lty = 2, col = "skyblue", lwd = 2)
    abline(h = -0.2, lty = 2, col = "skyblue", lwd = 2)

    plot(x = 1:13, y = data[[i]]$pacf, type = "n", ylim = c(-1, 1), main = paste("data", i,"PACF"), xlab = "lag", ylab = "PACF")
    abline(h = 0)
    segments(x0 = 1:13, y0 = 0, x1 = 1:13, y1 = data[[i]]$pacf)
    abline(h = 0.2, lty = 2, col = "skyblue", lwd = 2)
    abline(h = -0.2, lty = 2, col = "skyblue", lwd = 2)
}

In [31]:
24.75

[1] 24.75

### 4번 문제

In [ ]:
## data
ex7_3 = scan("ex7_3.txt")

#---------(1) Zt의 시계열 그림---------
ts.plot(ex7_3, ylab = expression(Z[t]), main = "Time Series Plot")
acf(ex7_3)

#---------(2) 1차 차분시계열의 시계열 그림---------
diff_z = diff(ex7_3)
ts.plot(diff_z, ylab = expression(Z[t]), main = "1차 차분")

diff14_z = diff(diff_z, 4)
ts.plot(diff14_z, ylab = expression(Z[t]), main = "1차 차분 + 계절주기가 4인 계절차분")

#---------(3) 계절차분만 진행한 시계열 그림---------
diff4_z = diff(ex7_3, 4)
ts.plot(diff4_z, ylab = expression(Z[t]), main = "계절주기가 4인 계절차분")

### 5번 문제

In [ ]:
## data
ex7_4 = scan("ex7_4.txt")

#---------(1) Zt의 시계열 그림---------
ts.plot(ex7_4, ylab = expression(Z[t]), main = "Time Series Plot")

#---------(2) 1차 차분시계열의 시계열 그림--------
diff_z = diff(ex7_4)
ts.plot(diff_z, ylab = expression(Z[t]), main = "1차 차분")

#---------(3) 계절차분된 시계열 그림---------
diff4_z = diff(ex7_4, 4)
ts.plot(diff4_z, ylab = expression(Z[t]), main = "계절주기가 4인 계절차분")

#---------(4) 계절차분 + 차분---------
diff14_z = diff(diff4_z)
ts.plot(diff14_z, ylab = expression(Z[t]), main = "계절주기가 4인 계절차분 + 1차 차분")